In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets list

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d deadskull7/fer2013
#!kaggle datasets download -d cfpb/us-consumer-finance-complaints
!ls

In [ ]:
import pandas as pd
data2 = pd.read_csv('/content/fer2013.zip', compression='zip', header=0, sep=',', quotechar='"')

In [ ]:
from zipfile import ZipFile
zip_file = ZipFile('/content/fer2013.zip')
#fields= ['product','consumer_complaint_narrative'] 
data=pd.read_csv(zip_file.open('fer2013.csv'))
data.head()

In [ ]:
!pip install keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers.normalization import BatchNormalization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [ ]:
labels = data.iloc[:,[0]].values

In [ ]:
pixels = data['pixels']

In [ ]:
Expressions = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
from keras.utils import to_categorical

In [ ]:
labels = to_categorical(labels, len(Expressions))

In [ ]:
# converting pixels to grayscale images of 48x48
images = np.array([np.fromstring(pixel, dtype='int', sep=' ')for pixel in pixels])
images = images/255.0
images = images.reshape(images.shape[0], 48, 48, 1).astype('float32')

In [ ]:
plt.imshow(images[0][:,:,0])
Expressions[labels[0][0]]

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(images,labels, test_size=0.2, random_state=0)

In [ ]:
train_labels

In [ ]:
train_images

In [ ]:
# CREATE THE MODEL
def convolutional_neural_network(classes):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(2,2), strides=(1,1), activation='relu', input_shape=(48, 48, 1)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, kernel_size=(2,2), strides=(1,1), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(128, kernel_size=(2,2), strides=(1,1), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(256, kernel_size=(2,2), strides=(1,1), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  
  model.add(Dense(256, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  
  model.add(Dense(512, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  
  model.add(Dense(classes, activation='softmax'))
  
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
  return model

In [ ]:
print(keras.__version__)

In [ ]:
classes = 7
model = convolutional_neural_network(classes)
model.summary()

In [ ]:
#train the CNN 
model.fit(train_images,train_labels,batch_size=105,epochs=30,verbose=2)

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# 1. Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import logging

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)

In [ ]:
# 2. Save the weights of the traines Keras model on the Google Drive
model.save("model.h5")
model_file = drive.CreateFile({"title":"model.h5"})
model_file.SetContentFile("model.h5")
model_file.Upload()

In [ ]:
# download to google drive
drive.CreateFile({'id': model_file.get('id')})